In [ ]:
import random
import os
import tkinter as tk
from tkinter import messagebox
import time

# 牌跟花色
cards = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
colors = ['Spade', 'Hearts', 'Diamond', 'Club']
deck_new = [[color , card ] for color in colors for card in cards]

# 畫面視窗
window = tk.Tk()
window.title('Game21')
window.geometry('600x400')
window.minsize(400, 300)
window.maxsize(700, 700)
window.config(bg='black') # 背景顏色
# 三個小視窗
div1 = tk.Frame(window,  width=400 , height=400 , bg='#7E5A10')
div2 = tk.Frame(window,  width=200 , height=200 , bg='#6E7E10')
div3 = tk.Frame(window,  width=200 , height=200 , bg='#263238')
# 放置 --對齊的工具
div1.grid(column=0, row=0, rowspan=2, padx=5, pady=5, sticky='nswe')
div2.grid(column=1, row=0, padx=5, pady=5, sticky='nswe')
div3.grid(column=1, row=1,  padx=5, pady=5, sticky='nswe')
# 比例 可以縮放
def define_layout(obj, cols=1, rows=1):
    
    def method(trg, col, row):
        
        for c in range(cols):    
            trg.columnconfigure(c, weight=1)
        for r in range(rows):
            trg.rowconfigure(r, weight=1)

    if type(obj)==list:        
        [ method(trg, cols, rows) for trg in obj ]
    else:
        trg = obj
        method(trg, cols, rows)
define_layout(window, cols=2, rows=2)
define_layout([div1, div2, div3])

# 文字變數 玩家
Vstr_player = tk.StringVar()
lbl_Player = tk.Label(div1, textvariable=Vstr_player, bg='yellow', fg='#263238', font=('Arial', 18))
lbl_Player.grid(column=0, row=0)
# 玩家分數
Vstr_player_score = tk.StringVar()
lbl_Player_score = tk.Label(div1, textvariable=Vstr_player_score, bg='yellow', fg='#263238', font=('Arial', 18))
lbl_Player_score.grid(column=0, row=1)
# 文字變數 莊家
Vstr_dealer = tk.StringVar()
lbl_Dealer = tk.Label(div2, textvariable=Vstr_dealer, bg='yellow', fg='#263238', font=('Arial', 18))
lbl_Dealer.grid(column=0, row=0)
# 莊家分數
Vstr_dealer_score = tk.StringVar()
lbl_Dealer_score = tk.Label(div2, textvariable=Vstr_dealer_score, bg='yellow', fg='#263238', font=('Arial', 18))
lbl_Dealer_score.grid(column=0, row=1)

class Player():
    def __init__(self, Name = 'Player'):
        self.hand = []
        self.name = Name

    def draw(self, deck):
        self.hand.append(deck.pop(random.randint(0, len(deck) - 1)))

    def showHand(self, hide_card = False):
        #給莊家用的 第一張牌蓋起來
        if hide_card:
            hand = self.name + '的手牌:\n' + '一張蓋牌和 ' + self.hand[0][0] + ' ' + self.hand[0][1]
            print(hand)
            return hand
        else:
            hand = self.name + '的手牌:' + '\n'
            for card in self.hand:
                hand+= card[0] + ' ' + card[1] + '\n'
            print(hand)
            return hand

    def countScore(self):
        score = 0
        ace = 0
        for card in self.hand:
            if card[1] == 'J' or card[1] == 'Q' or card[1] == 'K':
                score += 10
            elif card[1] == 'A':
                score += 1
                ace += 1
            else:
                score += int(card[1])

        if ace > 0 and score + 10 <= 21:
            score += 10
            ace -= 1
        #print(score)
        return score
    
addCardTK = tk.StringVar() # 判斷是否加牌的變數

def Play_game():
    print("遊戲開始")
    deck = deck_new.copy() # 新的一副牌
    random.shuffle(deck) # 洗牌
    btn_start.config(state='disabled') #遊戲開始變成不可按
    playerMe = Player('玩家')
    playerD = Player('莊家')
    playerMe.draw(deck)
    playerMe.draw(deck)
    playerD.draw(deck)
    playerD.draw(deck)
    
    while True: # 玩家回合
        # 顯示玩家跟莊家的分數
        Vstr_player.set(playerMe.showHand())
        Vstr_player_score.set('點數:'+str(playerMe.countScore()))
        Vstr_dealer.set(playerD.showHand(hide_card = True))
        Vstr_dealer_score.set("分數:?")

        addCardTK.set('s') #判斷是否加牌的變數
        print('SET_addCardTK')

        window.update() # 更新畫面
        time.sleep(0.5)

        if playerMe.countScore() == 21:
            Vstr_player_score.set('點數:'+str(playerMe.countScore()))
            break
        elif playerMe.countScore() > 21:
            print('玩家爆了')
            Vstr_player_score.set('玩家爆了!!點數:'+str(playerMe.countScore()))
            messagebox.showinfo('遊戲結束', '玩家爆了!莊家贏了')
            btn_start.config(state='normal') #變成可按
            return
        elif len(playerMe.hand) == 5:
            print("五張牌！玩家獲勝！")
            Vstr_player_score.set('五張牌！玩家獲勝！點數:'+str(playerMe.countScore()))
            messagebox.showinfo('遊戲結束', '玩家贏了')
            btn_start.config(state='normal') #變成可按
            return
        bt_Yes.wait_variable(addCardTK) # 等待按鈕的回應
        if addCardTK.get() == 'y':
            playerMe.draw(deck)
        elif addCardTK.get() == 'n':
            break
    
    print('--莊家回合--')

    while True: # 莊家回合
        Vstr_dealer.set(playerD.showHand()) #顯示莊家的手牌
        Vstr_dealer_score.set('點數:'+str(playerD.countScore())) #顯示莊家的分數
        # 計算抽到高分數的牌的風險
        cards_point = sum(1 for _, card in deck if card in ['9', '10', 'J', 'Q', 'K'])
        remaining_cards = len(deck)
        risk = cards_point / remaining_cards
        print(risk)
        window.update()
        if playerD.countScore() == 21:
            Vstr_dealer_score.set('點數:'+str(playerD.countScore()))
            break
        elif playerD.countScore() > 21:
            print('莊家爆了, 玩家贏了!')
            Vstr_dealer_score.set('莊家爆了, 玩家贏了!\n點數:'+str(playerD.countScore()))
            messagebox.showinfo('遊戲結束', '玩家贏了')
            btn_start.config(state='normal') #變成可按
            return
        elif len(playerD.hand)==5:
            print("五張牌！莊家獲勝！")
            Vstr_dealer_score.set('五張牌！莊家獲勝！\n點數:'+str(playerD.countScore()))
            messagebox.showinfo('遊戲結束', '莊家贏了')
            btn_start.config(state='normal') #變成可按
            return
        elif playerD.countScore() > 17:
            if any(card == 'A' for card in playerD.hand) and risk < 0.35:
                playerD.draw(deck)
            elif risk < 0.3:
                playerD.draw(deck)
            else:
                break
        elif playerD.countScore() <= 17:
            if risk < 0.45 or playerD.countScore() < 11:
                playerD.draw(deck)
            else:
                break
        
        window.update() # 更新畫面

    if playerD.countScore() > playerMe.countScore():
        print('莊家贏了')
        messagebox.showinfo('遊戲結束', '莊家贏了')
    elif playerD.countScore() < playerMe.countScore():
        print('玩家贏了')
        messagebox.showinfo('遊戲結束', '玩家贏了')
    elif playerD.countScore() == playerMe.countScore():
        print('平手')
        messagebox.showinfo('遊戲結束', '平手')

    btn_start.config(state='normal') #變成可按
    window.update()
    print('遊戲結束')
    return

# 按鈕
btn_start = tk.Button(div3, text='開始遊戲', font=('Arial', 18), command=Play_game)
btn_start.grid(column=0, row=0)
# 按鈕的回應
def bt_yes():
    addCardTK.set('y')
def bt_no():
    addCardTK.set('n')
# 抽牌按鈕
bt_Yes = tk.Button(div3, text='抽牌', bg='red', fg='white', command=bt_yes, font=('Arial', 24))
bt_Yes.grid(column=0, row=1)
# 不抽牌按鈕
bt_No = tk.Button(div3, text='不抽', bg='red', fg='white', command=bt_no, font=('Arial', 24))
bt_No.grid(column=1, row=1)

# 主程式
if __name__ == "__main__":
    #Play_game()
    window.mainloop() #開啟視窗
    

In [ ]:
# tkinter 介面 基本的物件 及用法
import tkinter as tk

window = tk.Tk()
window.title('Game21')
window.geometry('600x400')
window.minsize(400, 300)
window.maxsize(700, 700)
window.config(bg='black')
div_size = 200
img_size = div_size * 2
div1 = tk.Frame(window,  width=img_size , height=img_size , bg='#7E5A10')
div2 = tk.Frame(window,  width=div_size , height=div_size , bg='#6E7E10')
div3 = tk.Frame(window,  width=div_size , height=div_size , bg='#263238')

div1.grid(column=0, row=0, rowspan=2, padx=5, pady=5, sticky='nswe')
div2.grid(column=1, row=0, padx=5, pady=5, sticky='nswe')
div3.grid(column=1, row=1,  padx=5, pady=5, sticky='nswe')

def define_layout(obj, cols=1, rows=1):
    
    def method(trg, col, row):
        
        for c in range(cols):    
            trg.columnconfigure(c, weight=1)
        for r in range(rows):
            trg.rowconfigure(r, weight=1)

    if type(obj)==list:        
        [ method(trg, cols, rows) for trg in obj ]
    else:
        trg = obj
        method(trg, cols, rows)

define_layout(window, cols=2, rows=2)
define_layout([div1, div2, div3])

Vstr_player = tk.StringVar()  # 建立文字變數
Vstr_player.set('hello')      # 設定內容
lbl_Player = tk.Label(div1, textvariable=Vstr_player, bg='yellow', fg='#263238', font=('Arial', 18))
lbl_Player.grid(column=0, row=0)
Vstr_player.set('hello lbl_Player123')

lbl_Dealer = tk.Label(div2, text='lbl_Dealer', bg='yellow', fg='#263238', font=('Arial', 18))
lbl_Dealer.grid(column=0, row=0)

def bbt():
    Vstr_player.set('566655')
bt_Yes = tk.Button(div3, text='抽牌', bg='red', fg='white', command=bbt, font=('Arial', 24))
bt_Yes['activebackground'] = 'yellow'        # 按鈕被按下的背景顏色
bt_Yes['activeforeground'] = 'red'     # 按鈕被按下的文字顏色 ( 前景 )
bt_Yes.grid(column=0, row=1)

bt_No = tk.Button(div3, text='不抽', bg='red', fg='white', font=('Arial', 24))
bt_No['activebackground'] = 'yellow'
bt_No['activeforeground'] = 'red'
bt_No.grid(column=1, row=1)

bt_Start = tk.Button(div3, text='開始', bg='red', fg='white', font=('Arial', 24))
bt_Start['activebackground'] = 'yellow'
bt_Start['activeforeground'] = 'red'
bt_Start.grid(column=0, row=0)
bt_Start.grid_forget()
bt_Start.grid(column=0, row=0)
# define_layout(lbl_1)
# define_layout(bt_Yes)
# define_layout(bt_No)

window.mainloop()
